In [80]:
import requests
from bs4 import BeautifulSoup
import pdfplumber
import re

In [ ]:
def extract_text_from_pdf(pdf_url):
    """PDF URL에서 텍스트 추출"""
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()

        with open("./temp/temp.pdf", "wb") as f:
            f.write(response.content)

        with pdfplumber.open("./temp/temp.pdf") as pdf:
            text = "".join([page.extract_text() for page in pdf.pages])

        return text

    except requests.exceptions.RequestException as e:
        print(f"PDF 다운로드 에러: {e}")
        return None
    except Exception as e:
        print(f"PDF 텍스트 추출 중 에러 발생: {e}")
        return None

In [87]:
def extract_report_data(report_url):
    """리포트 URL에서 PDF URL을 추출하고, PDF 내용을 추출하여 반환"""
    try:
        pdf_url = report_url
        print(pdf_url)
        return extract_text_from_pdf(pdf_url)

    except requests.exceptions.RequestException as e:
        print(f"Request 에러: {e}")
        return None
    except Exception as e:
        print(f"리포트 데이터 추출 중 에러 발생: {e}")
        return None

In [88]:
def crawl_reports(base_url):
    """리포트 목록 페이지를 크롤링하여 리포트 정보 추출"""
    page_num = 1
    while True:
        url = f"{base_url}&page={page_num}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table", {"class": "type_1"})

        if not table:
            print("더 이상 리포트가 없습니다.")
            break

        rows = table.find_all("tr")[2:]  # 첫 두 행은 헤더이므로 제외

        if not rows:
            print("더 이상 리포트가 없습니다.")
            break

        for row in rows:
            cells = row.find_all("td")
            # print(cells)

            if not cells or all(cell.text.strip() == "" for cell in cells):
                continue

            company = cells[0].text.strip()
            # print("company:", company)
            title = cells[1].text.strip()
            # print("title:", title)
            report_url = cells[3].find("a")["href"]
            # print("report_url:", report_url)
            brokerage = cells[2].text.strip()
            # print("brokerage:", brokerage)
            date = cells[4].text.strip()
            # print("date:", date)

            report_content = extract_report_data(report_url)
            # print("report_content:", report_content)

            if report_content:
                report_data = {
                    "종목이름": company,
                    "종목코드": "035720",
                    "증권사": brokerage,
                    "제목": title,
                    "날짜": date,
                    "내용": report_content,
                }
                print(f"리포트 추출 성공: {report_data}")
            else:
                print(f"리포트 내용 추출 실패: {company}:{title}")
            break

        page_num += 1
        break

In [89]:
base_url = "https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&writeFromDate=&writeToDate=&searchType=itemCode&itemName=%C4%AB%C4%AB%BF%C0&itemCode=035720&x=58&y=13"
crawl_reports(base_url)

https://stock.pstatic.net/stock-research/company/18/20250214_company_561127000.pdf
리포트 추출 성공: {'종목이름': '카카오', '종목코드': '035720', '증권사': '유안타증권', '제목': '카톡의 외연 확장 기대', '날짜': '25.02.14', '내용': 'Company Report 2025.02.14\n카카오\n(035720)\n카톡의 외연 확장 기대\nBUY\n4Q24 Review (M)\n매출액 1조 9,591억원(YoY -2.0%), 영업이익 1,067억원(YoY -33.7%), 지배순손실\n1,318억원. 시장기대치 하회\n목표주가 53,000원 (M)\n직전 목표주가 53,000원\n광고/소비경기가 좋지 않음에도 불구 메시지광고 YoY +18% 성장 포함 톡광고 3,210억원\n현재주가 (2/13) 40,200원\nYoY +5%, 커머스 2,420억원 YoY +4%로 성장유지. 택시, 대리, 주차, 퀵의 고른 성장과\n금융서비스 최고 매출 기록하며 플랫폼 기타 4,031억원 YoY +22% 고성장 지속. 상승여력 32%\n반면 게임 1,630억원 YoY -30%, 에스엠 YoY +10%에도 뮤직(멜론 포함) 4,702억원 YoY\n시가총액 178,352억원\n-6%, 스토리(웹툰) 2,030억원 YoY -5%, 미디어 739억원 YoY -25% 등 컨텐츠 부문 역성\n총발행주식수 443,675,123주\n장 지속. 위메프 관련 대손상각비 315억원 반영으로 영업이익률 Y-Y 2.6%pt 감소\n60일 평균 거래대금 1,370억원\n2025년 전망 60일 평균 거래량 3,317,484주\n52주 고/저 59,200원 / 32,800원\n채팅 중심의 트래픽에서 다양한 컨텐츠를 소비할 수 있는 카톡으로 외연 확장 계획.\n외인지분율 27.23%\n[발견영역] 사용자 스스로 생성-소비하는 이미지, 동영상, 숏폼 등의 컨텐츠를 피드 형태로\n배당수익률 0.11%\n하반기 출시